# Título  Feature Engineering
## Autor: Jose Chelquer
## Fecha de última modificación: 24/10/2024
## Descripción:

Agrega features:
Sumas de variables (con o sin eliminación de las originales)
decilado de todas las variables de dinero (con o sin eliminación de las originales)
lag1, 2, 3 de todas
slope de algunas variables elegidas


## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [1]:
usar_gdrive=True    # Poner en true si se va a ejecutar con archivos de google drive en máquinas locales
#usar_gdrive=False

In [56]:
# A decilar
#         Va a decilar las columnas de dinero
a_decilar_eliminar_originales=True     # Eliminar las variables originales después de decilarlas
# Columnas a sumar
a_sumar_eliminar_originales=True        # Eliminar las variables sumadas después de crear las sumas
a_sumar = [
    [['ctarjeta_visa', 'ctarjeta_master'], 'ctarjeta'],
    [['ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones'], 'ctarjeta_transacciones'],
    [['mtarjeta_visa_consumo', 'mtarjeta_master_consumo'], 'mtarjeta_consumo'],
    [['cprestamos_personales', 'cprestamos_prendarios', 'cprestamos_hipotecarios'], 'cprestamos'],
    [['mprestamos_personales', 'mprestamos_prendarios', 'mprestamos_hipotecarios'], 'mprestamos'],
    [['cplazo_fijo', 'cinversion1', 'cinversion2'], 'cinversion'],
    [['mplazo_fijo_pesos', 'minversion1_pesos', 'minversion2'], 'minversion_pesos'],
    [['mplazo_fijo_dolares', 'minversion1_dolares'], 'minversion_dolares'],
    [['cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales'], 'cseguro'],
    [['ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'], 'ctarjeta_debitos_automaticos'],
    [['mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'], 'mtarjeta_debitos_automaticos'],
    [['cpagodeservicios', 'cpagomiscuentas'], 'cpagos'],
    [['mpagodeservicios', 'mpagomiscuentas'], 'mpagos'],
    [['Master_msaldopesos', 'Visa_msaldopesos'], 'Tarjeta_msaldopesos'],
    [['Master_msaldodolares', 'Visa_msaldodolares'], 'Tarjeta_msaldodolares'],
    [['Master_msaldototal', 'Visa_msaldototal'], 'Tarjeta_msaldototal'],
    [['Master_mconsumospesos', 'Visa_mconsumospesos'], 'Tarjeta_mconsumospesos'],
    [['Master_mconsumosdolares', 'Visa_mconsumosdolares'], 'Tarjeta_mconsumosdolares'],
    [['Master_mlimitecompra', 'Visa_mlimitecompra'], 'Tarjeta_mlimitecompra'],
    [['Master_madelantopesos', 'Visa_madelantopesos'], 'Tarjeta_madelantopesos'],
    [['Master_madelantodolares', 'Visa_madelantodolares'], 'Tarjeta_madelantodolares'],
    [['Master_mpagado', 'Visa_mpagado'], 'Tarjeta_mpagado'],
    [['Master_mpagospesos', 'Visa_mpagospesos'], 'Tarjeta_mpagospesos'],
    [['Master_mpagosdolares', 'Visa_mpagosdolares'], 'Tarjeta_mpagosdolares'],
    [['Master_mconsumototal', 'Visa_mconsumototal'], 'Tarjeta_mconsumototal'],
    [['Master_cadelantosefectivo', 'Visa_cadelantosefectivo'], 'Tarjeta_cadelantosefectivo'],
    [['Master_mpagominimo', 'Visa_mpagominimo'], 'Tarjeta_mpagominimo'],

]


## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [3]:
dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
#dataset_path = '~/buckets/b1/datasets'
dataset_file='competencia_01_sin_nan.csv'
#dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [4]:
output_file='competencia_02_aumentada.csv.gz'
duckdb_file='competencia_02.db'

## Procesos

### Paquetes necesarios

In [5]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [6]:
if usar_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import duckdb
import pandas as pd
import os



In [8]:
input_gz=dataset_file.endswith('.gz')
output_gz=output_file.endswith('.gz')

In [9]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
duckdb_path=os.path.join(dataset_path, duckdb_file)

conn = duckdb.connect(duckdb_path)


In [38]:
def do_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  print(resultado)

def get_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  return resultado

In [41]:
# Leer el CSV y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
if input_gz:
  sql = f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}', COMPRESSION='GZIP')"
else:
  sql = f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}')"
print (sql)
conn.execute(sql)


CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_sin_nan.csv')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Listas de de variables

In [42]:
# Variables dinero originales (después se le agregarán las creadas en base a ellas)
dinero=[
 'mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheques_depositados_rechazados',
 'mcheques_emitidos_rechazados',
 'matm',
 'matm_other',
 'Master_mfinanciacion_limite',
 'Master_msaldototal',
 'Master_msaldopesos',
 'Master_msaldodolares',
 'Master_mconsumospesos',
 'Master_mconsumosdolares',
 'Master_mlimitecompra',
 'Master_madelantopesos',
 'Master_madelantodolares',
 'Master_mpagado',
 'Master_mpagospesos',
 'Master_mpagosdolares',
 'Master_mconsumototal',
 'Master_mpagominimo',
 'Visa_mfinanciacion_limite',
 'Visa_msaldototal',
 'Visa_msaldopesos',
 'Visa_msaldodolares',
 'Visa_mconsumospesos',
 'Visa_mconsumosdolares',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_madelantodolares',
 'Visa_mpagado',
 'Visa_mpagospesos',
 'Visa_mpagosdolares',
 'Visa_mconsumototal',
 'Visa_mpagominimo',
]


In [43]:
# Crea macro de suma_sin_null
do_sql('CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);')



CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);
[]


In [51]:
# Obtiene lista de columnas
def columnas(tabla):
  sql=f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = '{tabla}';
  """
  lista=get_sql(sql)
  lista=[elemento[0] for elemento in lista]

  return lista


# Prepara las sumas sin null
def agregables_a(a_sumar, lista):
  # Seleccionan de todos los campo#
  # aquellos cuyos sumandos están en la lista
  a_agregar=[]
  for caso in a_sumar:
    if set(caso[0]).issubset(set(lista)):
      a_agregar.append(caso[1])
  return a_agregar

def crear_suma_sin_null_un_caso(caso):
  sumandos=caso[0]
  resultado=caso[1]
# Construye la línea
  linea=sumandos[0]

  sumandos=caso[0]
  suma=caso[1]
  linea=f'{sumandos[0]}'
  for sumando in sumandos[1:]:
    linea=f'suma_sin_null({sumando}, {linea})'
  linea=f'{linea} as {resultado}'
  return linea

def lista_suma_sin_null (a_sumar):
  lineas=[]
  for caso in a_sumar:
    lineas.append(crear_suma_sin_null_un_caso(caso))
  return lineas

def sql_sumar_sin_null(a_sumar, tabla):
  lista=lista_suma_sin_null(a_sumar)
  texto_con_saltos = "\n,".join(lista)
  sql=f'create or replace table {tabla} as \n  select *, {texto_con_saltos}\nfrom {tabla};'
  return sql

def lista_de_sumandos(a_sumar):
  return[elemento for sublista in a_sumar for elemento in sublista[0]]

def sql_eliminar_columnas(columnas, tabla):
    # Unir las columnas en un formato adecuado para el SQL
    sql = '; \n'.join([f"ALTER TABLE {tabla} DROP COLUMN {col}" for col in columnas])
    return sql

## Campos suma

In [45]:
# Agrega los nuevos campos dinero
dinero.extend(agregables_a(a_sumar, dinero))

# Crea kas variables suma
sql=sql_sumar_sin_null(a_sumar, 'competencia_02')
do_sql(sql)

# Elimina las originales
if a_sumar_eliminar_originales:
  sql=sql_eliminar_columnas(lista_de_sumandos(a_sumar), 'competencia_02')
  do_sql(sql)

create or replace table competencia_02 as 
  select *, suma_sin_null(ctarjeta_master, ctarjeta_visa) as ctarjeta
,suma_sin_null(ctarjeta_master_transacciones, ctarjeta_visa_transacciones) as ctarjeta_transacciones
,suma_sin_null(mtarjeta_master_consumo, mtarjeta_visa_consumo) as mtarjeta_consumo
,suma_sin_null(cprestamos_hipotecarios, suma_sin_null(cprestamos_prendarios, cprestamos_personales)) as cprestamos
,suma_sin_null(mprestamos_hipotecarios, suma_sin_null(mprestamos_prendarios, mprestamos_personales)) as mprestamos
,suma_sin_null(cinversion2, suma_sin_null(cinversion1, cplazo_fijo)) as cinversion
,suma_sin_null(minversion2, suma_sin_null(minversion1_pesos, mplazo_fijo_pesos)) as minversion_pesos
,suma_sin_null(minversion1_dolares, mplazo_fijo_dolares) as minversion_dolares
,suma_sin_null(cseguro_accidentes_personales, suma_sin_null(cseguro_vivienda, suma_sin_null(cseguro_auto, cseguro_vida))) as cseguro
,suma_sin_null(ctarjeta_master_debitos_automaticos, ctarjeta_visa_debitos_aut

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(981946,)]
ALTER TABLE competencia_02 DROP COLUMN ctarjeta_visa; 
ALTER TABLE competencia_02 DROP COLUMN ctarjeta_master; 
ALTER TABLE competencia_02 DROP COLUMN ctarjeta_visa_transacciones; 
ALTER TABLE competencia_02 DROP COLUMN ctarjeta_master_transacciones; 
ALTER TABLE competencia_02 DROP COLUMN mtarjeta_visa_consumo; 
ALTER TABLE competencia_02 DROP COLUMN mtarjeta_master_consumo; 
ALTER TABLE competencia_02 DROP COLUMN cprestamos_personales; 
ALTER TABLE competencia_02 DROP COLUMN cprestamos_prendarios; 
ALTER TABLE competencia_02 DROP COLUMN cprestamos_hipotecarios; 
ALTER TABLE competencia_02 DROP COLUMN mprestamos_personales; 
ALTER TABLE competencia_02 DROP COLUMN mprestamos_prendarios; 
ALTER TABLE competencia_02 DROP COLUMN mprestamos_hipotecarios; 
ALTER TABLE competencia_02 DROP COLUMN cplazo_fijo; 
ALTER TABLE competencia_02 DROP COLUMN cinversion1; 
ALTER TABLE competencia_02 DROP COLUMN cinversion2; 
ALTER TABLE competencia_02 DROP COLUMN mplazo_fijo_pesos; 
ALTER TA

In [46]:
sql="""
CREATE OR REPLACE MACRO division_segura (a, b) AS (
  CASE
    WHEN b IS NULL THEN 0
    WHEN a IS NULL THEN 0
    WHEN b = 0 THEN 0
    WHEN a = 0 THEN 0
    ELSE a / b
  END
  )
"""
do_sql(sql)



CREATE OR REPLACE MACRO division_segura (a, b) AS (
  CASE
    WHEN b IS NULL THEN 0
    WHEN a IS NULL THEN 0
    WHEN b = 0 THEN 0
    WHEN a = 0 THEN 0
    ELSE a / b
  END
  )

[]


In [ ]:
# grabar competencia aumentada con features
def grabar_aumentada():
  if output_gz:
    output_file_path=os.path.join(dataset_path, output_file)
    sql=(f"COPY competencia_02 TO '{output_file_path}' (FORMAT CSV, HEADER, COMPRESSION 'GZIP')")
  else:
    sql=(f"COPY competencia_02 TO '{output_file_path}' (FORMAT CSV, HEADER)")
  print (f"Se grabó {output_file_path}")


## Grupos de variables

In [ ]:
%%sql --no-execute
select
    *,
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5

    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,ratio_comisiones_recursos,ratio_comisiones_recursos_1,foto_mes_1,numero_de_cliente_1,cliente_antiguedad_1,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,0.022461,0.0,202101,249221109,300,1,0.947564,0.947947,4,10
1,249221109,202102,1,0,0,61,301,1002.92,18337.34,1312.91,...,0.012355,0.0,202102,249221109,301,2,0.947831,0.948213,4,10
2,249221109,202103,1,0,0,61,302,769.74,18596.58,1662.56,...,0.024973,0.0,202103,249221109,302,3,0.948291,0.948670,4,10
3,249221109,202104,1,0,0,62,303,1532.79,19935.00,1899.52,...,0.009814,0.0,202104,249221109,303,4,0.948638,0.949016,4,10
4,249221109,202105,1,0,0,62,304,1804.14,20862.15,2693.91,...,0.094253,0.0,202105,249221109,304,5,0.948950,0.949327,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,0.000000,0.0,202106,1598444941,1,1,0.000000,0.001553,1,1
981942,1598471047,202106,1,0,0,28,1,95.20,95.20,0.00,...,0.000000,0.0,202106,1598471047,1,1,0.000000,0.001553,1,1
981943,1598515897,202106,0,0,0,30,1,72.08,72.08,0.00,...,0.000000,0.0,202106,1598515897,1,1,0.000000,0.001553,1,1
981944,1598517059,202106,0,0,0,24,1,0.00,0.00,0.00,...,0.000000,0.0,202106,1598517059,1,1,0.000000,0.001553,1,1


## Decilar


In [53]:
decilar=dinero
columnas_tabla=columnas('competencia_02')
decilar = [elemento for elemento in dinero if elemento in columnas_tabla]
orden=""
orden=orden+"create or replace TABLE competencia_02 as\n"
orden+=f"select *\n"
for campo in decilar:
  orden+=f",ntile(10) over (partition by foto_mes order by {campo}) as {campo}_n10\n"
orden+=f"from competencia_02\n"


do_sql(orden)




  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
create or replace TABLE competencia_02 as
select *
,ntile(10) over (partition by foto_mes order by mrentabilidad) as mrentabilidad_n10
,ntile(10) over (partition by foto_mes order by mrentabilidad_annual) as mrentabilidad_annual_n10
,ntile(10) over (partition by foto_mes order by mcomisiones) as mcomisiones_n10
,ntile(10) over (partition by foto_mes order by mactivos_margen) as mactivos_margen_n10
,ntile(10) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_n10
,ntile(10) over (partition by foto_mes order by mcuenta_corriente_adicional) as mcuenta_corriente_adicional_n10
,ntile(10) over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_n10
,ntile(10) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_n10
,ntile(10) over (partition by foto_mes order by mcaja_ahorro_adicional) as mcaja_ahorro_adicional_n10
,ntile(10) over (partitio

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(981946,)]


In [57]:
if a_decilar_eliminar_originales:
  sql=sql_eliminar_columnas(decilar, 'competencia_02')
  do_sql(sql)

ALTER TABLE competencia_02 DROP COLUMN mrentabilidad; 
ALTER TABLE competencia_02 DROP COLUMN mrentabilidad_annual; 
ALTER TABLE competencia_02 DROP COLUMN mcomisiones; 
ALTER TABLE competencia_02 DROP COLUMN mactivos_margen; 
ALTER TABLE competencia_02 DROP COLUMN mpasivos_margen; 
ALTER TABLE competencia_02 DROP COLUMN mcuenta_corriente_adicional; 
ALTER TABLE competencia_02 DROP COLUMN mcuenta_corriente; 
ALTER TABLE competencia_02 DROP COLUMN mcaja_ahorro; 
ALTER TABLE competencia_02 DROP COLUMN mcaja_ahorro_adicional; 
ALTER TABLE competencia_02 DROP COLUMN mcaja_ahorro_dolares; 
ALTER TABLE competencia_02 DROP COLUMN mcuentas_saldo; 
ALTER TABLE competencia_02 DROP COLUMN mautoservicio; 
ALTER TABLE competencia_02 DROP COLUMN mpayroll; 
ALTER TABLE competencia_02 DROP COLUMN mpayroll2; 
ALTER TABLE competencia_02 DROP COLUMN mcuenta_debitos_automaticos; 
ALTER TABLE competencia_02 DROP COLUMN mcajeros_propios_descuentos; 
ALTER TABLE competencia_02 DROP COLUMN mtarjeta_visa_descu

## Lags

In [61]:
orden=""
orden=orden+"create or replace TABLE competencia_02 as\n"
orden+=f"select *\n"
for campo in columnas('competencia_02'):
  for lag in [1, 2, 3]:
    orden+=f",lag({campo}, {lag}) over (partition by numero_de_cliente order by foto_mes) as {campo}_lag{lag}\n"
orden+=f"from competencia_02\n"

do_sql(orden)




  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
create or replace TABLE competencia_02 as
select *
,lag(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes) as numero_de_cliente_lag1
,lag(numero_de_cliente, 2) over (partition by numero_de_cliente order by foto_mes) as numero_de_cliente_lag2
,lag(numero_de_cliente, 3) over (partition by numero_de_cliente order by foto_mes) as numero_de_cliente_lag3
,lag(foto_mes, 1) over (partition by numero_de_cliente order by foto_mes) as foto_mes_lag1
,lag(foto_mes, 2) over (partition by numero_de_cliente order by foto_mes) as foto_mes_lag2
,lag(foto_mes, 3) over (partition by numero_de_cliente order by foto_mes) as foto_mes_lag3
,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as active_quarter_lag1
,lag(active_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as active_quarter_lag2
,lag(active_quarter, 3) over (partition by nu

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(981946,)]


## Otros agregados:
promedios de valores en meses
cambios en cant de tarjetas u otros.
comparar un valor con el mínimo, máximo o promedio de los últimos meses,


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql --no-execute
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


Skipping execution...

Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
a_slopear=[
  'mrentabilidad'
, 'cproductos'
, 'mpayroll'
, 'mpayroll2'
, 'cpayroll'
, 'mcuentas_saldo'
, 'ccuenta_debitos_automaticos'
, 'ccajas_transacciones'
, 'ccajas_consultas'
, 'ccajas_depositos'
, 'ccajas_extracciones'
, 'Master_status'
, 'Visa_status'
, 'ctarjeta'
, 'ctarjeta_transacciones'
, 'cprestamos'
, 'cinversion'
, 'cseguro'
, 'ctarjeta_debitos_automaticos'
, 'mcuentas_saldo'
, 'msaldos_e_inversion'
]
a_slopear= [col for col in a_slopear if col in columnas('competencia_02')]


In [1]:
orden=""
orden+=f"select *\n"
for campo in a_slopear:
  orden+=f",regr_slope({campo}, foto_mes) over ventana_3 as {campo}_slope_3\n"
orden+=f"from competencia_01\n"
orden+=f"window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)\n"
do_sql(orden)



NameError: name 'ver_slope' is not defined

In [ ]:
grabar_aumentada()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))